In [8]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker --quiet

In [9]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [11]:
model_id, model_version = "huggingface-textgeneration1-gpt-j-6b", "*"

In [12]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

training_instance_type = "ml.g4dn.12xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

In [13]:
# Sample training data is available in this bucket
data_bucket = f"<data-s3-bucket>"

training_dataset_s3_path = f"s3://{data_bucket}/train/"


output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-tg-train"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

In [15]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epoch"] = "30"
# hyperparameters["per_device_train_batch_size"] = "8"
hyperparameters["auto_find_batch_size"] = True
print(hyperparameters)

{'epoch': '30', 'learning_rate': '6e-06', 'per_device_train_batch_size': '2', 'per_device_eval_batch_size': '8', 'warmup_ratio': '0.1', 'instruction_tuned': 'False', 'train_from_scratch': 'False', 'fp16': 'True', 'bf16': 'False', 'evaluation_strategy': 'steps', 'eval_steps': '20', 'gradient_accumulation_steps': '2', 'logging_steps': '10', 'weight_decay': '0.2', 'load_best_model_at_end': 'True', 'max_train_samples': '-1', 'max_val_samples': '-1', 'seed': '10', 'max_input_length': '-1', 'validation_split_ratio': '0.2', 'train_data_split_seed': '0', 'preprocessing_num_workers': 'None', 'max_steps': '-1', 'gradient_checkpointing': 'True', 'early_stopping_patience': '3', 'early_stopping_threshold': '0.0', 'adam_beta1': '0.9', 'adam_beta2': '0.999', 'adam_epsilon': '1e-08', 'max_grad_norm': '1.0', 'label_smoothing_factor': '0', 'logging_first_step': 'False', 'logging_nan_inf_filter': 'True', 'save_strategy': 'steps', 'save_steps': '500', 'save_total_limit': '1', 'dataloader_drop_last': 'Fals

In [16]:
from sagemaker.tuner import ContinuousParameter

# Use AMT for tuning and selecting the best model
use_amt = False

# Define objective metric, based on which the best model will be selected.
amt_metric_definitions = {
    "metrics": [{"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"}],
    "type": "Minimize",
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.0001, 0.001, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 2

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

metric_definitions = [
    {"Name": "train:loss", "Regex": "'loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:runtime", "Regex": "'eval_runtime': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:samples_per_second", "Regex": "'eval_samples_per_second': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:eval_steps_per_second", "Regex": "'eval_steps_per_second': ([0-9]+\.[0-9]+)"},
]


# Create SageMaker Estimator instance
tg_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=metric_definitions,
)

if use_amt:
    hp_tuner = HyperparameterTuner(
        tg_estimator,
        amt_metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        amt_metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=amt_metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"train": training_dataset_s3_path, "validation": validation_dataset_s3_path})
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    tg_estimator.fit(
        {"train": training_dataset_s3_path}, logs=True
    )

INFO:sagemaker:Creating training-job with name: jumpstart-example-huggingface-textgener-2023-09-05-02-25-16-230


2023-09-05 02:25:16 Starting - Starting the training job...
2023-09-05 02:25:42 Starting - Preparing the instances for training......
2023-09-05 02:26:32 Downloading - Downloading input data..........................................................................
2023-09-05 02:39:00 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-05 02:39:02,256 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-05 02:39:02,289 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-05 02:39:02,298 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-09-05 02:39:02,300 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-09-05 02:39:04,457 sagemaker-training-toolkit INFO     Installing dependencies from requi

In [19]:
from sagemaker import TrainingJobAnalytics

if use_amt:
    training_job_name = hp_tuner.best_training_job()
else:
    training_job_name = tg_estimator.latest_training_job.job_name

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df.head(10)


,timestamp,metric_name,value
0,0.0,train:loss,2.5852
1,360.0,train:loss,2.3801
2,720.0,train:loss,2.1456
3,1020.0,train:loss,1.7910
4,1380.0,train:loss,1.4953
5,1740.0,train:loss,1.1247
6,2100.0,train:loss,0.8130
7,2400.0,train:loss,0.5367
8,2760.0,train:loss,0.2545
9,3120.0,train:loss,0.1662


In [ ]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)

# deploy the Model. TODO
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

In [20]:
inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

endpoint_name_after_finetune = name_from_base(f"jumpstart-example-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = (hp_tuner if use_amt else tg_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name_after_finetune,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py39.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.g5.12xlarge.
INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2023-09-05-07-28-45-903
INFO:sagemaker:Creating endpoint-config with name jumpstart-example-huggingface-textgener-2023-09-05-07-28-45-903
INFO:sagemaker:Creating endpoint with name jumpstart-example-huggingface-textgener-2023-09-05-07-28-45-903


-----------!

In [5]:
import json
import boto3


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


def parse_response_multiple_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

In [29]:
def pre_trained_model(question):
    parameters = {
        "max_length": 250,
        "num_return_sequences": 1,
        "top_k": 150,
        "top_p": 0.8,
        "do_sample": True,
        "temperature": 1,
    }
    
    endpoint_name = "pre-trained-model-endpoint"

    res_gpt_before_finetune = []
    
    payload = {"text_inputs": f"{question}:", **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_before_finetune.append(generated_texts)
    print("Pre-trained model response:")
    print("\n")
    print(generated_texts)
    print("-------------------------------")
    

In [30]:
def fine_tuned_model(question):
    parameters = {
        "max_length": 250,
        "num_return_sequences": 1,
        "top_k": 150,
        "top_p": 0.8,
        "do_sample": True,
        "temperature": 1,
    }

    endpoint_name = "fine-tuned-model-endpoint"
    res_gpt_before_finetune = []
    
    payload = {"text_inputs": f"{question}:", **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_before_finetune.append(generated_texts)
    print("Fine-tuned model response:")
    print("\n")
    print(generated_texts)
    print("-------------------------------")

In [31]:
question = "What motivates Captain Ahab's relentless pursuit of Moby Dick throughout the novel?"

pre_trained_model(question)
fine_tuned_model(question)

Pre-trained model response:


What motivates Captain Ahab's relentless pursuit of Moby Dick throughout the novel?:


Ahab has driven his family from their home. His family, and most of his crew have died at sea or died as prisoners of war. Ahab, alone in a universe without God, becomes the devil himself. He is the only one who sees himself as the only one in the universe.




Ahab is a character that is a man of few friends, if any at all. Most of the time he is a solitary and angry person, except for the time he spends with the crew of the Pequod. It is this crew that is his best friend, and family.

Ahab has been given the name of white whale and is a great enemy of white whale. He is seen as the only one to ever find white whale in his nature.
Ahab and the white whale are one of the most important character in Moby Dick. The white whale is a metaphor for God and for white whale represents Ahab's hatred for God, as he says in the novel that the white whale killed him as the devil.

A

In [37]:
question = "Analyze the character of Queequeg and his role in the novel named Moby Dick. What does he represent?"

pre_trained_model(question)
fine_tuned_model(question)

Pre-trained model response:


Analyze the character of Queequeg and his role in the novel named Moby Dick. What does he represent?: what does he represent to his character and the novel? How does this reflect upon the concept of the American Dream?: what does this concept mean in the context of the book? Does the book or any of the characters seem to be like what we are all too familiar with? Discuss with other students and learn more about the concept of "The American Dream" and what the book is about. Themes of the book? Discuss with your class what you learned from the novel and how this relates to your personal life and how it may relate to you.
-------------------------------
Fine-tuned model response:


Analyze the character of Queequeg and his role in the novel named Moby Dick. What does he represent?: Is he good or evil?

“I will now proceed to analyze the character of Queequeg, and his role in the book named Moby Dick. I will begin by saying that he represents an ancient and h

In [86]:
question = "Analyze the character named Ahab and his role from the novel named Moby Dick?"

pre_trained_model(question)
fine_tuned_model(question)

Pre-trained model response:


Analyze the character named Ahab and his role from the novel named Moby Dick?: The Whale, Part 2

How would you evaluate his role in the novel Moby Dick

How would you interpret his role
His role in the novel?
Can you think about it as
A character
In his role and what role
How would you analyze his role in the novel?
Would you like to give me your opinion?
Is it true that he is
It's been discussed in the novel?
Novel?
Yes, I mean
It's been discussed?
You know, it's in the novel, and it's been
What role that has been discussed?
In the novel, you have to analyze?
What are the points that are discussed
in the novel?
Yes, but you can say
In the novel
In the novel
Yes, you can discuss it, but
You can have a different opinion.

A, but you can
think about it, can you can discuss it, but you can have a different opinion about it, so

But if I can have different opinion about it?
Can you have a different


What is
-------------------------------
Fine-tuned model re

In [ ]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()